# Predicting Newsletter Subscription on a Minecraft Research Server Using Player and Session Data

**Background**
Email newsletters are a low-cost, high-impact channel for keeping research-server players engaged over the long term. On the PLAI Minecraft server run by UBC Computer Science, some participants opt in to a game-related newsletter while others do not. Knowing which demographic traits and in-game behaviours drive that decision helps the research team (1) focus recruitment on the most receptive audiences and (2) design targeted retention campaigns that nudge undecided players toward deeper involvement.

**Question**
Which player characteristics and play-style metrics best predict whether a participant subscribes to the server’s newsletter?
By modelling newsletter opt-in as a binary outcome, we aim to identify actionable features—such as total play time, preferred login hours, or self-reported experience level—that distinguish subscribers from non-subscribers.

**Data Description**
This analysis combines information from two datasets:

players.csv – one row per player (n ≈ 196).
Key variables:
- subscribe (boolean) – target flag indicating newsletter opt-in
- experience (categorical) – Pro, Veteran, Regular, etc.
- gender, age, country, operating-system strings
- hashedEmail (unique anonymised ID)

sessions.csv – one row per game session (≈ 1 535 records).
Selected variables:
- hashedEmail – foreign key to players.csv
- start_time, end_time – session boundaries (ISO datetime)
- original_start_time, original_end_time – millisecond timestamps

Session records will be aggregated per player to derive behavioural predictors—e.g., total minutes played, mean session length, weekday/weekend ratio, and most frequent hour-of-day. After merging with players.csv, the resulting feature matrix will feed a classification model (logistic regression and tree-based methods) to quantify which factors most influence newsletter subscription.

**Load Packages**

In [221]:
### Run this cell before continuing. 
library(tidyverse)
library(lubridate)
library(tidymodels)

**load raw data**

In [222]:
players  <- read_csv("data/players.csv",
                     col_types = cols(.default = col_guess()))
sessions <- read_csv("data/sessions.csv",
                     col_types = cols(.default = col_guess()))

head(players)
head(sessions)

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [223]:
# convert the subscribe variable to factor
players <- players |>
    mutate(subscribe = as_factor(subscribe))

In [224]:
# each row is unique
nrow(players)
n_distinct(players$hashedEmail)

[1] 196

[1] 196

In [225]:

# Exploring Data to see the proportion of True and False in subscribe variable. Baseline
num_obs <- nrow(players)
players |>
  group_by(subscribe) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100
  )


subscribe,count,percentage
<fct>,<int>,<dbl>
FALSE,52,26.53061
TRUE,144,73.46939


In [226]:
# mutate variables to proper type, and correct value
sessions_mutate <- sessions_clean |>
  mutate(
    start_dt = dmy_hm(start_time, tz = "UTC"),
    end_dt   = dmy_hm(end_time,   tz = "UTC"),
    dur_min  = as.numeric(difftime(end_dt, start_dt, units = "mins")),
    wday_lbl = wday(start_dt, label = TRUE),
    hour24   = hour(start_dt)
  )


In [227]:
# mutate sessions to get more meaningful data
sessions_processed <- sessions_mutate |>
  group_by(hashedEmail) |> 
  summarise(
    tot_minutes   = sum(dur_min,   na.rm = TRUE),
    n_sessions    = n(),                              # simple count
    avg_session_min   = mean(dur_min,  na.rm = TRUE),
    evening_prop  = mean(hour24 %in% 18:23),          # 0-1 ratio
    weekend_prop  = mean(wday_lbl %in% c("Sat","Sun"))
  )

In [228]:

# merge with players.csv
combined_data <- players |>
  inner_join(sessions_processed, by = "hashedEmail") |>
  mutate(
    subscribe = as.factor(subscribe),           
  )

In [229]:
#checks how many hashedEmail created at least one session. Matches data in combined_data
nrow(combined_data)
n_distinct(sessions$hashedEmail)
nrow(players)

[1] 125

[1] 125

[1] 196

In [230]:
set.seed(42)                            
data_split <- initial_split(
  combined_data,
  prop   = 0.70,
  strata = subscribe
)
train_data <- training(data_split)
test_data  <- testing(data_split)

knn_rec <- recipe(subscribe ~., data = train_data) |>
  step_rm(hashedEmail, name) |>
  step_other(gender, threshold = 0.02) |>
  step_novel(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(), one_hot = TRUE) |>
  step_impute_median(Age) |>
  step_zv(all_predictors()) |>
  step_normalize(all_numeric_predictors())


knn_spec <- nearest_neighbor(neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

folds <- vfold_cv(train_data, v = 5, strata = subscribe)


# we use 25 as our threshold for k.
k_grid <- tibble(neighbors = seq(1, 11, by = 2))

# finish the workflow
knn_wf <- workflow() |>
          add_recipe(knn_rec) |>
          add_model(knn_spec) |>
          tune_grid(resamples = folds, grid = k_grid) |>
          collect_metrics()

accuracies <- knn_wf |>
  filter(.metric == "accuracy")

#sort the list by highest accruacy.
best_k <- knn_wf |>           
  filter(.metric == "accuracy") |>
  arrange(desc(mean)) |>
  slice(1) |>                    
  pull(neighbors)

# best_k is 23.
best_k


# retrieve all the variable that we will find accuracy
vars <- setdiff(names(train_data), c("subscribe", "hashedEmail", "name"))

knn_spec_fixed <- nearest_neighbor(neighbors = best_k) |>
                set_engine("kknn") |>
                set_mode("classification")

one_var_knn_acc <- function(var){
  mini_train <- train_data |> select(subscribe, all_of(var))
  mini_test  <- test_data  |> select(subscribe, all_of(var))

  if (is.numeric(train_data[[var]])) {
    rec <- recipe(subscribe ~ ., data = mini_train) |>
           step_normalize(all_numeric_predictors())
  } else {
    rec <- recipe(subscribe ~ ., data = mini_train) |>
           step_other(all_nominal_predictors(), threshold = 0.02) |>
           step_novel(all_nominal_predictors()) |>
           step_dummy(all_nominal_predictors(), one_hot = TRUE) |>
           step_zv(all_predictors())
  }

  fit <- workflow() |>
        add_recipe(rec) |>
        add_model(knn_spec_fixed) |>
        fit(mini_train)

  acc <- accuracy(
           bind_cols(predict(fit, mini_test),
                     mini_test |> select(subscribe)),
           truth = subscribe, estimate = .pred_class
         ) |>
        pull(.estimate)

  tibble(variable = var, accuracy = acc)
}

scores <- map_dfr(vars, one_var_knn_acc) |>
        arrange(desc(accuracy))
print(scores)

baseline_acc <- test_data |>
  count(subscribe) |>
  summarise(baseline = max(n) / sum(n)) |>
  pull(baseline)

baseline_acc


knn_rec <- recipe(subscribe ~avg_session_min + tot_minutes + experience, data = train_data) |>
  step_novel(all_nominal_predictors()) |>
  step_dummy(all_nominal_predictors(), one_hot = TRUE) |>
  step_zv(all_predictors()) |>
  step_normalize(all_numeric_predictors())


knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(knn_rec) |>
  add_model(knn_spec) |>
  fit(data = train_data)


test_preds <- predict(knn_fit, test_data) |>
              bind_cols(test_data |> select(subscribe))

acc_tbl <- accuracy(test_preds,
                    truth    = subscribe,
                    estimate = .pred_class)

acc_tbl$.estimate           # just the numeric value

[1] 11

# A tibble: 9 × 2
  variable        accuracy
  <chr>              <dbl>
1 avg_session_min    0.711
2 tot_minutes        0.658
3 Age                0.526
4 played_hours       0.474
5 n_sessions         0.421
6 experience         0.395
7 evening_prop       0.368
8 weekend_prop       0.368
9 gender             0.289


[1] 0.7368421

[1] 0.7368421